In [1]:
from __future__ import annotations

import os 

from torch.utils.data import random_split

from mmpfn.datasets.cbis_ddsm import CBISDDSMDataset

import os 
import torch 
import numpy as np 
import pandas as pd

from sklearn.metrics import accuracy_score, log_loss, roc_auc_score, root_mean_squared_error

from mmpfn.models.tabpfn_v2 import TabPFNClassifier
from mmpfn.models.dino_v2.models.vision_transformer import vit_base
from mmpfn.models.tabpfn_v2.constants import ModelInterfaceConfig
from mmpfn.models.tabpfn_v2.preprocessing import PreprocessorConfig
from mmpfn.scripts_finetune.finetune_tabpfn_main import fine_tune_tabpfn

/home/wall/works/research/MultiModalPFN/mmpfn/models/dino_v2/layers/swiglu_ffn.py:51: UserWarning: xFormers is not available (SwiGLU)
  warnings.warn("xFormers is not available (SwiGLU)")
/home/wall/works/research/MultiModalPFN/mmpfn/models/dino_v2/layers/attention.py:33: UserWarning: xFormers is not available (Attention)
  warnings.warn("xFormers is not available (Attention)")
/home/wall/works/research/MultiModalPFN/mmpfn/models/dino_v2/layers/block.py:40: UserWarning: xFormers is not available (Block)
  warnings.warn("xFormers is not available (Block)")


In [2]:
# data_path = os.path.join(os.getenv('HOME'), "workspace/works/tabular_image/MultiModalPFN/mmpfn/data/cbis_ddsm")
data_path = os.path.join(os.getenv('HOME'), "works/research/MultiModalPFN/mmpfn/data/cbis_ddsm")

kind = 'mass'  # mass calc
image_type = 'all' # all full crop roi
test_dataset = CBISDDSMDataset(data_path=data_path, data_name=f'csv/{kind}_case_description_test_set.csv', kind=kind, image_type=image_type)
# _ = test_dataset.get_images()
# _ = test_dataset.get_embeddings(mode='test')
train_dataset = CBISDDSMDataset(data_path=data_path, data_name=f'csv/{kind}_case_description_train_set.csv', kind=kind, image_type=image_type)
# _ = train_dataset.get_images()
# _ = train_dataset.get_embeddings(mode='train')

In [3]:
accuracy_scores = []
for seed in range(5):
    torch.manual_seed(seed)
    # np.random.seed(seed)
    # print(f"Finetuning with seed: {seed}")
    
    X_train = train_dataset.x
    y_train = train_dataset.y
    X_test = test_dataset.x
    y_test = test_dataset.y

    for i in range(X_train.shape[1]):
        col = X_train[:, i]
        col[np.isnan(col)] = np.nanmin(col) - 1
    for i in range(X_test.shape[1]):
        col = X_test[:, i]
        col[np.isnan(col)] = np.nanmin(col) - 1

    torch.cuda.empty_cache()

    save_path_to_fine_tuned_model = "./finetuned_tabpfn_cbis_mass.ckpt"
    
    fine_tune_tabpfn(
        # path_to_base_model="auto",
        save_path_to_fine_tuned_model=save_path_to_fine_tuned_model,
        # Finetuning HPs
        time_limit=60,
        finetuning_config={"learning_rate": 0.00001, "batch_size": 1, "max_steps": 100},
        validation_metric="log_loss",
        # Input Data
        X_train=pd.DataFrame(X_train),
        y_train=pd.Series(y_train),
        categorical_features_index=None,
        device="cuda",  # use "cpu" if you don't have a GPU
        task_type="binary",
        # Optional
        show_training_curve=False,  # Shows a final report after finetuning.
        logger_level=0,  # Shows all logs, higher values shows less
    )

    # disables preprocessing at inference time to match fine-tuning
    no_preprocessing_inference_config = ModelInterfaceConfig(
        FINGERPRINT_FEATURE=False,
        PREPROCESS_TRANSFORMS=[PreprocessorConfig(name='none')]
    )

    # Evaluate on Test Data
    model_finetuned = TabPFNClassifier(
        model_path=save_path_to_fine_tuned_model,
        inference_config=no_preprocessing_inference_config,
        ignore_pretraining_limits=True,
    )

    clf_finetuned = model_finetuned.fit(X_train, y_train)
    acc_score = accuracy_score(y_test, clf_finetuned.predict(X_test))
    print("accuracy_score (Finetuned):", acc_score)
    accuracy_scores.append(acc_score)

Fine-tuning Steps:   1%|          | 1/100 [00:00<?, ?it/s][2025-09-17 23:52:28,965] INFO - 
Optimizer step skipped due to NaNs/infs in grad scaling.
Fine-tuning Steps:  23%|██▎       | 23/100 [00:06<00:18,  4.06it/s, Best Val. Loss=0.276, Best Val. Score=-0.276, Training Loss=0.247, Val. Loss=0.276, Patience=29, Utilization=0, Grad Norm=3.76][2025-09-17 23:52:35,305] INFO - 
Optimizer step skipped due to NaNs/infs in grad scaling.
Fine-tuning Steps: 101it [00:24,  4.11it/s, Best Val. Loss=0.249, Best Val. Score=-0.249, Training Loss=0.287, Val. Loss=0.259, Patience=-48, Utilization=0, Grad Norm=3.81]                         
[2025-09-17 23:52:52,870] INFO - Initial Validation Loss: 0.6337409752581222 Best Validation Loss: 0.2490369253392985 Total Steps: 101 Best Step: 68 Total Time Spent: 25.512139320373535


accuracy_score (Finetuned): 0.7142857142857143


Fine-tuning Steps:   1%|          | 1/100 [00:00<?, ?it/s][2025-09-17 23:52:53,726] INFO - 
Optimizer step skipped due to NaNs/infs in grad scaling.
Fine-tuning Steps:  23%|██▎       | 23/100 [00:05<00:21,  3.59it/s, Best Val. Loss=0.275, Best Val. Score=-0.275, Training Loss=0.247, Val. Loss=0.275, Patience=29, Utilization=0, Grad Norm=3.76][2025-09-17 23:52:59,415] INFO - 
Optimizer step skipped due to NaNs/infs in grad scaling.
Fine-tuning Steps: 101it [00:24,  4.12it/s, Best Val. Loss=0.249, Best Val. Score=-0.249, Training Loss=0.284, Val. Loss=0.259, Patience=-48, Utilization=0, Grad Norm=4.16]                         
[2025-09-17 23:53:17,796] INFO - Initial Validation Loss: 0.6337409752581222 Best Validation Loss: 0.24926094292862372 Total Steps: 101 Best Step: 68 Total Time Spent: 24.51719880104065


accuracy_score (Finetuned): 0.7142857142857143


Fine-tuning Steps:   1%|          | 1/100 [00:00<?, ?it/s][2025-09-17 23:53:18,516] INFO - 
Optimizer step skipped due to NaNs/infs in grad scaling.
Fine-tuning Steps:  23%|██▎       | 23/100 [00:05<00:20,  3.76it/s, Best Val. Loss=0.276, Best Val. Score=-0.276, Training Loss=0.247, Val. Loss=0.276, Patience=29, Utilization=0, Grad Norm=3.74][2025-09-17 23:53:24,270] INFO - 
Optimizer step skipped due to NaNs/infs in grad scaling.
Fine-tuning Steps: 101it [00:24,  4.05it/s, Best Val. Loss=0.248, Best Val. Score=-0.248, Training Loss=0.282, Val. Loss=0.258, Patience=-48, Utilization=0, Grad Norm=3.77]                         
[2025-09-17 23:53:43,010] INFO - Initial Validation Loss: 0.6337409752581222 Best Validation Loss: 0.24790622825402506 Total Steps: 101 Best Step: 67 Total Time Spent: 24.916123390197754


accuracy_score (Finetuned): 0.708994708994709


Fine-tuning Steps:   1%|          | 1/100 [00:00<?, ?it/s][2025-09-17 23:53:43,738] INFO - 
Optimizer step skipped due to NaNs/infs in grad scaling.
Fine-tuning Steps:  23%|██▎       | 23/100 [00:05<00:21,  3.66it/s, Best Val. Loss=0.276, Best Val. Score=-0.276, Training Loss=0.247, Val. Loss=0.276, Patience=29, Utilization=0, Grad Norm=3.75][2025-09-17 23:53:49,410] INFO - 
Optimizer step skipped due to NaNs/infs in grad scaling.
Fine-tuning Steps: 101it [00:24,  4.08it/s, Best Val. Loss=0.248, Best Val. Score=-0.248, Training Loss=0.285, Val. Loss=0.258, Patience=-48, Utilization=0, Grad Norm=3.67]                         
[2025-09-17 23:54:08,027] INFO - Initial Validation Loss: 0.6337409752581222 Best Validation Loss: 0.24809013480246075 Total Steps: 101 Best Step: 69 Total Time Spent: 24.712132453918457


accuracy_score (Finetuned): 0.7142857142857143


Fine-tuning Steps:   1%|          | 1/100 [00:00<?, ?it/s][2025-09-17 23:54:08,784] INFO - 
Optimizer step skipped due to NaNs/infs in grad scaling.
Fine-tuning Steps:  23%|██▎       | 23/100 [00:06<00:21,  3.65it/s, Best Val. Loss=0.275, Best Val. Score=-0.275, Training Loss=0.247, Val. Loss=0.275, Patience=29, Utilization=0, Grad Norm=3.74][2025-09-17 23:54:14,816] INFO - 
Optimizer step skipped due to NaNs/infs in grad scaling.
Fine-tuning Steps: 101it [00:25,  3.96it/s, Best Val. Loss=0.249, Best Val. Score=-0.249, Training Loss=0.286, Val. Loss=0.257, Patience=-48, Utilization=0, Grad Norm=3.78]                         
[2025-09-17 23:54:33,808] INFO - Initial Validation Loss: 0.6337409752581222 Best Validation Loss: 0.24939014855316755 Total Steps: 101 Best Step: 70 Total Time Spent: 25.47745132446289


accuracy_score (Finetuned): 0.7116402116402116


In [4]:
# get mean and std of accuracy scores
mean_accuracy = np.mean(accuracy_scores)
std_accuracy = np.std(accuracy_scores)
print("Mean Accuracy:", mean_accuracy)
print("Std Accuracy:", std_accuracy)

Mean Accuracy: 0.7126984126984127
Std Accuracy: 0.0021164021164021387
